In [1]:
import pandas as pd
import re

In [2]:

pd.set_option('display.max_columns', None)

data = pd.read_csv('trail1.csv')

data.sample(10)

,Video Title,Video URL,Youtube Creator,YT Creator URL,Is Ad,Time
12112,Alan Walker - LIVE @ the Bergen Aquarium (Gold...,https://www.youtube.com/watch?v=LZ2FKB30fsw,Alan Walker,https://www.youtube.com/channel/UCJrOtniJ0-NWz...,False,"12 Jul 2021, 11:57:44 IST"
9353,Katy Perry - Firework (Official Music Video),https://www.youtube.com/watch?v=QGJuMBdaqIw,KatyPerryVEVO,https://www.youtube.com/channel/UC-8Q-hLdECwQm...,False,"24 Apr 2022, 21:56:19 IST"
1672,four funny African Snails eat Pumpkin 😍 #shorts,https://www.youtube.com/watch?v=4TRvKPXbbv4,Весёлая Улитка,https://www.youtube.com/channel/UC23zcaO_k99Jb...,False,"7 Mar 2023, 20:35:23 IST"
7730,CSS Tutorial: Creating Transitions in CSS | We...,https://www.youtube.com/watch?v=k4Dr0PJKidI,CodeWithHarry,https://www.youtube.com/channel/UCeVMnSShP_Ivi...,False,"21 Jun 2022, 08:13:39 IST"
2351,Taarak Mehta Ka Ooltah Chashmah - Episode 555 ...,https://www.youtube.com/watch?v=psPQOlhfFs4,Sony SAB,https://www.youtube.com/channel/UC6-F5tO8uklgE...,False,"13 Dec 2022, 23:50:14 IST"
9749,EMQX Cloud in 90 seconds,https://www.youtube.com/watch?v=0v6ZMBdqmIM,NaN,NaN,True,"27 Mar 2022, 09:50:09 IST"
3598,Cheese Maggi Recipe | Street Style Cheesy Masa...,https://www.youtube.com/watch?v=kKEW8bgtGn0,The Terrace Kitchen,https://www.youtube.com/channel/UC2bgPCeViczOg...,False,"15 Oct 2022, 18:54:19 IST"
11039,Why Edutainers Recommend Teuida,https://www.youtube.com/watch?v=OxKoqMpll5M,NaN,NaN,True,"9 Feb 2022, 18:29:35 IST"
9384,#9 Django tutorials | Model View Template in D...,https://www.youtube.com/watch?v=GGkFg52Ot5o,Telusko,https://www.youtube.com/channel/UC59K-uG2A5ogw...,False,"24 Apr 2022, 18:52:01 IST"
4495,Reality CHECK 😂,https://www.youtube.com/watch?v=BSYEWIBkm80,Victoria Pfeifer,https://www.youtube.com/channel/UCQBzn-XGeA4sq...,False,"2 Oct 2022, 18:13:58 IST"


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12624 entries, 0 to 12623
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Video Title      12624 non-null  object
 1   Video URL        12624 non-null  object
 2   Youtube Creator  10908 non-null  object
 3   YT Creator URL   10909 non-null  object
 4   Is Ad            12624 non-null  bool  
 5   Time             12624 non-null  object
dtypes: bool(1), object(5)
memory usage: 505.6+ KB


In [4]:
history = data.copy()
history.sample(3)

,Video Title,Video URL,Youtube Creator,YT Creator URL,Is Ad,Time
8032,SPY×FAMILY - Preview of Episode 11 [English Sub],https://www.youtube.com/watch?v=FucuKckRJBI,Muse Asia,https://www.youtube.com/channel/UCGbshtvS9t-8C...,False,"15 Jun 2022, 15:01:45 IST"
9086,Naruto Most Savage Moments pt 3,https://www.youtube.com/watch?v=kz7bmllQ7Pg,ghetto Hokage,https://www.youtube.com/channel/UCHxjBf1utfcKN...,False,"10 May 2022, 14:41:18 IST"
230,ALL TIMES Yuta Okkotsu is Mentioned & Shown in...,https://www.youtube.com/watch?v=7QcXcEkUJeQ,Niyn,https://www.youtube.com/channel/UCoW8QeekquXlI...,False,"16 Jul 2023, 23:24:36 IST"


transforming time col ( as there are some other string present with time in cell)

In [5]:
def ref_time(st):
    date_time_pattern = r'\b(?:0?[1-9]|[1-2][0-9]|3[01])\s(?:Jan|Feb|Mar|Apr|May|Jun|Jul|Aug|Sep|Oct|Nov|Dec)\s\d{4},\s\d{2}:\d{2}:\d{2}\s\w+\b'
    st = st.strip()
    matches = re.findall(date_time_pattern, st)

    if matches:
        extracted_date_time = matches[0]
        if extracted_date_time[0].isdigit() and extracted_date_time[1].isdigit():
            if int(extracted_date_time[0:2]) > 30:
                return extracted_date_time[1:]
        return extracted_date_time

    months = {
        'Jan' : 31, 'Feb' : [28, 29], 'Mar': 31, 'Apr': 30, 'May': 31, 'Jun': 30, 'Aug':31, ' Se': 30, 'Oct':31, 'Nov':30, 'Dec':31}

    if len(st) >= 26 and st[-26].isdigit():
        if st[-25].isdigit():
            cur = st[-24:-21]
            date = int(st[-26:-24])
            if cur != 'Feb':
                if  date <= months[cur]:
                        return st[-26:]
            else:
                yr = int(st[-18:-14])
                if (yr % 100 == 0 and yr % 4 == 0) or (yr % 100 != 0 and yr % 4 == 0):
                    if date <= months['Feb'][1]:
                        return st[-26:]
                else:
                    return st[-26:]
    return st[-25:]



history['Time'] = history['Time'].apply(ref_time)

In [6]:
history

,Video Title,Video URL,Youtube Creator,YT Creator URL,Is Ad,Time
0,How to Download your YouTube Data,https://www.youtube.com/watch?v=5CGgqJWh7wo,Buzz2day Tech,https://www.youtube.com/channel/UCSaA50dPTKM4X...,False,"23 Jul 2023, 19:20:17 IST"
1,"Farhan Akhtar ke bol, Farhan Akhtar ki awaaz m...",https://www.youtube.com/watch?v=fP-WXYk1wVc,NaN,NaN,True,"23 Jul 2023, 19:17:10 IST"
2,🎙️ From Syria to Revolutionising Men's Groomin...,https://www.youtube.com/watch?v=ekcgdD_q7bU,Sonny Sangha,https://www.youtube.com/channel/UCqeTj_QAnNlmt...,False,"23 Jul 2023, 19:15:09 IST"
3,MOVING TO DUBAI | Vlog #0,https://www.youtube.com/watch?v=hP1V5wehdEU,Sonny Sangha,https://www.youtube.com/channel/UCqeTj_QAnNlmt...,False,"23 Jul 2023, 19:15:00 IST"
4,Python Project to Scrape YouTube using YouTube...,https://www.youtube.com/watch?v=SwSbnmqk3zY,techTFQ,https://www.youtube.com/channel/UCnz-ZXXER4jOv...,False,"23 Jul 2023, 19:12:48 IST"
...,...,...,...,...,...,...
12619,10$ Drum Vs 1000$ Drum,https://www.youtube.com/watch?v=FnY_R0ZUGGM,Big Marvel,https://www.youtube.com/channel/UCgI8VzlDGsHCp...,False,"1 Aug 2020, 17:30:17 IST"
12620,Kokila Ben | First World Problems | Dialogue w...,https://www.youtube.com/watch?v=SjhBeIz1tDI,Yashraj Mukhate,https://www.youtube.com/channel/UCBZQAxIn4Mxc2...,False,"1 Aug 2020, 17:28:14 IST"
12621,"Mehul Choksi, Zakir Naik gave donation to Raji...",https://www.youtube.com/watch?v=9JbrEW1A0xY,Bharatiya Janata Party,https://www.youtube.com/channel/UCrwE8kVqtIUVU...,False,"1 Aug 2020, 16:21:46 IST"
12622,Prime Minister Narendra Modi's Mann Ki Baat wi...,https://www.youtube.com/watch?v=OgafNvdpGHI,Narendra Modi,https://www.youtube.com/channel/UC1NF71EwP41Vd...,False,"1 Aug 2020, 14:39:49 IST"


In [7]:
history['Time'].unique()

array(['23 Jul 2023, 19:20:17 IST', '23 Jul 2023, 19:17:10 IST',
       '23 Jul 2023, 19:15:09 IST', ..., '1 Aug 2020, 16:21:46 IST',
       '1 Aug 2020, 14:39:49 IST', '16 Jun 2020, 20:17:32 IST'],
      dtype=object)

In [8]:
history.sample(5)

,Video Title,Video URL,Youtube Creator,YT Creator URL,Is Ad,Time
9201,Speed: Brave Browser,https://www.youtube.com/watch?v=F_wOwuko2NM,NaN,NaN,True,"8 May 2022, 18:53:26 IST"
2040,Your beauty never ever scared me || Nezuko & T...,https://www.youtube.com/watch?v=nesLzDRvzhw,imthesan,https://www.youtube.com/channel/UCHr5gYvzISpll...,False,"27 Jan 2023, 21:40:48 IST"
1187,Top Down parser | Types of Parser | Compiler D...,https://www.youtube.com/watch?v=a4qdbCQJhyA,Education 4u,https://www.youtube.com/channel/UCKS34cSMNaXay...,False,"21 Apr 2023, 18:50:49 IST"
9026,He didn't knew that he was against the fastest...,https://www.youtube.com/watch?v=uc0ibnXRlCU,OTAKU VIBES,https://www.youtube.com/channel/UCHOxj9OSRWRZ1...,False,"12 May 2022, 09:10:05 IST"
5738,CAUGHT PASSING NOTES IN CLASS,https://www.youtube.com/watch?v=AhkzeWh0XBU,Will McIlroy,https://www.youtube.com/channel/UCbU-zdJ1JKg2Q...,False,"17 Aug 2022, 14:26:37 IST"


In [9]:
history.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12624 entries, 0 to 12623
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Video Title      12624 non-null  object
 1   Video URL        12624 non-null  object
 2   Youtube Creator  10908 non-null  object
 3   YT Creator URL   10909 non-null  object
 4   Is Ad            12624 non-null  bool  
 5   Time             12624 non-null  object
dtypes: bool(1), object(5)
memory usage: 505.6+ KB


In [11]:
history['Time']

0        23 Jul 2023, 19:20:17 IST
1        23 Jul 2023, 19:17:10 IST
2        23 Jul 2023, 19:15:09 IST
3        23 Jul 2023, 19:15:00 IST
4        23 Jul 2023, 19:12:48 IST
                   ...            
12619     1 Aug 2020, 17:30:17 IST
12620     1 Aug 2020, 17:28:14 IST
12621     1 Aug 2020, 16:21:46 IST
12622     1 Aug 2020, 14:39:49 IST
12623    16 Jun 2020, 20:17:32 IST
Name: Time, Length: 12624, dtype: object

Most Watched Video

In [12]:
video_counts = history['Video URL'].value_counts()

most_common_video_url = video_counts.idxmax()
highest_count = video_counts.max()
most_common_video_rows = history[history['Video URL'] == most_common_video_url]

print("Most Common Video URL:", most_common_video_url)
print("Video Title: ", most_common_video_rows.iloc[0]['Video Title'])
print("Count:", highest_count)

Most Common Video URL: https://www.youtube.com/watch?v=xGPjonxrUYc
Video Title:  Hardwell Live at Ultra Music Festival Miami 2017
Count: 31


In [13]:
top_5_videos = video_counts.nlargest(5)

top_5_rows = history[history['Video URL'].isin(top_5_videos.index)]

unique_videos = top_5_rows.drop_duplicates(subset=['Video URL', 'Video Title', 'Youtube Creator'])

print(unique_videos[['Youtube Creator', 'Video Title']])

         Youtube Creator                                        Video Title
491             Hardwell   Hardwell Live at Ultra Music Festival Miami 2017
512             Hardwell   Hardwell LIVE at Ultra Music Festival Miami 2018
739      Nightblue Music  Warriyo - Mortals (feat. Laura Brehm) (ELPORT ...
7214          Capital FM  Taylor Swift - ‘...Ready For It?’ (Live At Cap...
8628  LeeandLie (AmaLee)  Sword Art Online - "Crossing Field" (REMIX) | ...


Most watched videos of yt creator

In [14]:
creator_counts = history['Youtube Creator'].value_counts()

most_watched_yt_creator = creator_counts.idxmax()
highest_count = creator_counts.max()

print("Most watched yt creator:", most_watched_yt_creator)
print("Video Count:", highest_count)

Most watched yt creator: TheMermaidscales
Video Count: 345


In [15]:
top_5_creators = creator_counts.nlargest(5)


print(top_5_creators)

TheMermaidscales    345
CodeWithHarry       337
Gate Smashers       168
Capital FM          119
Aditya Verma        109
Name: Youtube Creator, dtype: int64


In [16]:
top_videos_by_top_creators = history[history['Youtube Creator'].isin(top_5_creators.index)]

top_videos_by_creator_title = top_videos_by_top_creators.groupby(['Youtube Creator', 'Video Title']).size()

top_videos_by_creator_title = top_videos_by_creator_title.reset_index(name='Watch Count')
top_5_videos_by_creator = top_videos_by_creator_title.groupby('Youtube Creator').apply(lambda x: x.nlargest(5, 'Watch Count')).reset_index(drop=True)
print(top_5_videos_by_creator)

     Youtube Creator                                        Video Title  \
0       Aditya Verma                           4 01Knapsack Memoization   
1       Aditya Verma                   Delete Middle Element of a Stack   
2       Aditya Verma                              13 Unbounded Knapsack   
3       Aditya Verma    15  Coin change problem: Maximum number of ways   
4       Aditya Verma           19  Longest common subsequence Recursive   
5         Capital FM  Taylor Swift - ‘...Ready For It?’ (Live At Cap...   
6         Capital FM  Zara Larsson - 'Lush Life' (Live At The Summer...   
7         Capital FM  Ariana Grande - 'Bang Bang' (Live At Capitals ...   
8         Capital FM  Taylor Swift - You Need to Calm Down (Live at ...   
9         Capital FM  Zara Larsson - ‘Lush Life’  (Live at Capital’s...   
10     CodeWithHarry  Component Structure of our NewsMonkey React Ap...   
11     CodeWithHarry  Creating our first react app using create-reac...   
12     CodeWithHarry  Fix